## 최종 결과물 : Catboost(Public : 5.1721157857, Private : 8.4329062881)


## 1.Library & Data Load

In [2]:
from google.colab import drive
drive.mount('/content/drive')      

Mounted at /content/drive


In [ ]:
!pip install catboost
!pip install optuna

In [4]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

import random
import optuna
from optuna.samplers import TPESampler

In [6]:
path='/data/235867_╣░╖∙ └п┼ы╖о ┐╣├° ░ц┴°┤ы╚╕/'

In [7]:
train = pd.read_csv(path+'train_df.csv', encoding='CP949')
test = pd.read_csv(path+'test_df.csv', encoding='CP949')

submission = pd.read_csv(path+'sample_submission.csv')

## 2. Data Preprocess

송하인 격자공간고유번호와 수하인격자공간고유번호가 연속형 자료로 취급하면 안된다고 생각하였습니다.

격자공간고유번호의 16자리 중 자릿수마다 담고 있는 정보가 다를 것이라 생각하였고,
데이터를 탐색해본 결과 1 ~ 5자리, 6 ~ 9자리, 10, 11 ~ 16자리가 가지고 있는 정보가 다를 것이라 생각했습니다.

그리고 송하인 격자공간고유번호의 unique 수는 꽤 되지만 수하인 격자공간고유번호의 unique 수는 얼마 되지 않았습니다. 

따라서 송하인 격자공간고유번호는 1 ~ 5, 6 ~ 9, 10, 11 ~ 16자릿수로 나누고
수하인 격자공간고유번호는 자릿수 별로 변수를 생성하였습니다.

총 22개의 설명변수로 이루어진 데이터로 변환하였습니다.

In [8]:
def numround(number, digit):
  num=[]
  while(number!=0):
    num.append(number % 10)
    number = number //10

  return int(num[-digit])

In [9]:
for i in tqdm(range(16)):
  train[f'SEND_SPG_INNB_{i+1}'] = 0
  train[f'REC_SPG_INNB_{i+1}'] = 0
  test[f'SEND_SPG_INNB_{i+1}'] = 0
  test[f'REC_SPG_INNB_{i+1}'] = 0
  for j in range(train.shape[0]):
    train.loc[j,f'SEND_SPG_INNB_{i+1}']=numround(train.loc[j,'SEND_SPG_INNB'],i+1)
    train.loc[j,f'REC_SPG_INNB_{i+1}']=numround(train.loc[j,'REC_SPG_INNB'],i+1)

  for j in range(test.shape[0]):
    test.loc[j,f'SEND_SPG_INNB_{i+1}']=numround(test.loc[j,'SEND_SPG_INNB'],i+1)
    test.loc[j,f'REC_SPG_INNB_{i+1}']=numround(test.loc[j,'REC_SPG_INNB'],i+1)


100%|██████████| 16/16 [10:29<00:00, 39.31s/it]


In [10]:
train['SEND_SPG_INNB_1~5']=train['SEND_SPG_INNB_1']+train['SEND_SPG_INNB_2']+train['SEND_SPG_INNB_3']+train['SEND_SPG_INNB_4']+train['SEND_SPG_INNB_5']
train['SEND_SPG_INNB_6~9']=train['SEND_SPG_INNB_6']+train['SEND_SPG_INNB_7']+train['SEND_SPG_INNB_8']+train['SEND_SPG_INNB_9']
train['SEND_SPG_INNB_10']=train['SEND_SPG_INNB_10']
train['SEND_SPG_INNB_11~16']=train['SEND_SPG_INNB_11']+train['SEND_SPG_INNB_12']+train['SEND_SPG_INNB_13']+train['SEND_SPG_INNB_14']+train['SEND_SPG_INNB_15']+train['SEND_SPG_INNB_16']

test['SEND_SPG_INNB_1~5']=test['SEND_SPG_INNB_1']+test['SEND_SPG_INNB_2']+test['SEND_SPG_INNB_3']+test['SEND_SPG_INNB_4']+test['SEND_SPG_INNB_5']
test['SEND_SPG_INNB_6~9']=test['SEND_SPG_INNB_6']+test['SEND_SPG_INNB_7']+test['SEND_SPG_INNB_8']+test['SEND_SPG_INNB_9']
test['SEND_SPG_INNB_10']=test['SEND_SPG_INNB_10']
test['SEND_SPG_INNB_11~16']=test['SEND_SPG_INNB_11']+test['SEND_SPG_INNB_12']+test['SEND_SPG_INNB_13']+test['SEND_SPG_INNB_14']+test['SEND_SPG_INNB_15']+test['SEND_SPG_INNB_16']


In [11]:
train.index=train['index']
test.index=test['index']
train.drop(['REC_SPG_INNB','SEND_SPG_INNB','SEND_SPG_INNB_1','SEND_SPG_INNB_2','SEND_SPG_INNB_3','SEND_SPG_INNB_4','SEND_SPG_INNB_5','SEND_SPG_INNB_6','SEND_SPG_INNB_7',
            'SEND_SPG_INNB_8','SEND_SPG_INNB_9','SEND_SPG_INNB_11','SEND_SPG_INNB_12','SEND_SPG_INNB_13','SEND_SPG_INNB_14','SEND_SPG_INNB_15','SEND_SPG_INNB_16','index'],axis=1,inplace=True)
test.drop(['REC_SPG_INNB','SEND_SPG_INNB','SEND_SPG_INNB_1','SEND_SPG_INNB_2','SEND_SPG_INNB_3','SEND_SPG_INNB_4','SEND_SPG_INNB_5','SEND_SPG_INNB_6','SEND_SPG_INNB_7',
            'SEND_SPG_INNB_8','SEND_SPG_INNB_9','SEND_SPG_INNB_11','SEND_SPG_INNB_12','SEND_SPG_INNB_13','SEND_SPG_INNB_14','SEND_SPG_INNB_15','SEND_SPG_INNB_16','index'],axis=1,inplace=True)

In [12]:
for col in test.columns:
  train[col]=train[col].astype('category')
  test[col]=test[col].astype('category')

In [13]:
#Optuna용 Train셋
X = train.drop(['INVC_CONT'],axis=1)
y = train['INVC_CONT']
X_test = test.copy()

In [14]:
X.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,REC_SPG_INNB_1,REC_SPG_INNB_2,REC_SPG_INNB_3,REC_SPG_INNB_4,REC_SPG_INNB_5,REC_SPG_INNB_6,REC_SPG_INNB_7,REC_SPG_INNB_8,REC_SPG_INNB_9,SEND_SPG_INNB_10,REC_SPG_INNB_10,REC_SPG_INNB_11,REC_SPG_INNB_12,REC_SPG_INNB_13,REC_SPG_INNB_14,REC_SPG_INNB_15,REC_SPG_INNB_16,SEND_SPG_INNB_1~5,SEND_SPG_INNB_6~9,SEND_SPG_INNB_11~16
index,,,,,,,,,,,,,,,,,,,,,,
0,패션의류,상의,5,0,1,1,0,0,0,2,2,4,0,0,4,6,3,0,0,13,1,12
1,생활/건강,반려동물,5,0,1,1,0,0,0,1,7,9,8,0,3,7,3,0,0,10,0,8
2,패션의류,기타패션의류,5,0,1,1,0,0,0,2,6,0,5,0,9,1,4,0,0,10,3,13
3,식품,농산물,5,0,1,1,0,0,0,3,1,2,5,0,8,7,4,0,0,16,0,7
4,식품,가공식품,5,0,1,1,0,0,0,1,7,1,7,0,5,1,2,0,0,13,2,11


In [15]:
y.head()

index
0     3
1     3
2     9
3    10
4     3
Name: INVC_CONT, dtype: int64

In [16]:
X_test.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,REC_SPG_INNB_1,REC_SPG_INNB_2,REC_SPG_INNB_3,REC_SPG_INNB_4,REC_SPG_INNB_5,REC_SPG_INNB_6,REC_SPG_INNB_7,REC_SPG_INNB_8,REC_SPG_INNB_9,SEND_SPG_INNB_10,REC_SPG_INNB_10,REC_SPG_INNB_11,REC_SPG_INNB_12,REC_SPG_INNB_13,REC_SPG_INNB_14,REC_SPG_INNB_15,REC_SPG_INNB_16,SEND_SPG_INNB_1~5,SEND_SPG_INNB_6~9,SEND_SPG_INNB_11~16
index,,,,,,,,,,,,,,,,,,,,,,
32000,식품,농산물,1,1,6,5,0,0,0,0,2,3,1,0,9,7,2,0,0,9,4,14
32001,식품,농산물,1,1,5,4,5,0,0,0,0,4,2,0,6,6,4,0,0,9,4,8
32002,식품,농산물,4,1,3,9,0,0,0,1,0,5,2,0,1,3,2,0,0,9,2,5
32003,식품,농산물,4,2,2,1,0,0,0,0,4,5,0,0,9,3,4,0,0,9,2,5
32004,식품,농산물,2,7,2,6,0,0,0,0,0,8,4,0,1,7,1,0,0,9,8,6


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 0 to 31999
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   DL_GD_LCLS_NM        32000 non-null  category
 1   DL_GD_MCLS_NM        32000 non-null  category
 2   INVC_CONT            32000 non-null  int64   
 3   REC_SPG_INNB_1       32000 non-null  category
 4   REC_SPG_INNB_2       32000 non-null  category
 5   REC_SPG_INNB_3       32000 non-null  category
 6   REC_SPG_INNB_4       32000 non-null  category
 7   REC_SPG_INNB_5       32000 non-null  category
 8   REC_SPG_INNB_6       32000 non-null  category
 9   REC_SPG_INNB_7       32000 non-null  category
 10  REC_SPG_INNB_8       32000 non-null  category
 11  REC_SPG_INNB_9       32000 non-null  category
 12  SEND_SPG_INNB_10     32000 non-null  category
 13  REC_SPG_INNB_10      32000 non-null  category
 14  REC_SPG_INNB_11      32000 non-null  category
 15  REC_SPG_INNB_12    

## 3. 모델링

Optuna로 Random Search를 통해 Catboost 최적의 파라미터를 사용하였습니다.
objective 함수의 param에 파라미터를 넣고, 구간을 넣으면 랜덤한 값으로 학습되며
rmse값이 반환되는 함수입니다. "trial"에 반복 횟수를 작성하면 됩니다.

Catboost 특성상 학습이 오래 걸리기 때문에 최적의 파라미터를 찾아 cat_param로 정의하였습니다.

(아래코드는 AIBoo님의 신용카드 사용자 연체 예측 AI 경진대회 [Private 8위 0.66203] | TYKIM | Catboost 코드를 참고하여 수정하였습니다.)

In [18]:
# def objective(trial):
#   param = {
#       "random_state":42,
#       'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.05),
#       'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
#       "n_estimators":trial.suggest_int("n_estimators", 500, 5000),
#       "max_depth":trial.suggest_int("max_depth", 4, 16),
#       'random_strength' :trial.suggest_int('random_strength', 0, 100),
#       "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
#       "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
#       "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
#       "max_bin": trial.suggest_int("max_bin", 200, 500),
#       'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#   }
#   X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)
#   cat_features = range(X_test.shape[1])
#   cat = CatBoostRegressor(**param)
#   cat.fit(X_train, y_train,
#           eval_set=[(X_train, y_train), (X_valid,y_valid)],
#           early_stopping_rounds=35,cat_features=cat_features,
#           verbose=100)
#   cat_pred = cat.predict(X_valid)
#   rmse = np.sqrt(mean_squared_error(y_valid, cat_pred))

#   return rmse

In [19]:
# sampler = TPESampler(seed=42)
# study = optuna.create_study(
#     study_name = 'cat_parameter_opt',
#     direction = 'minimize',
#     sampler = sampler,
# )

# study.optimize(objective, n_trials=10)
# print("Best Score:",study.best_value)
# print("Best trial",study.best_trial.params)

In [20]:
cat_param={'learning_rate': 0.018272261776066247, 
           'bagging_temperature': 63.512210106407046, 
           'n_estimators': 3794, 
           'max_depth': 11, 
           'random_strength': 15, 
           'colsample_bylevel': 0.49359671220172163, 
           'l2_leaf_reg': 1.7519275289243016e-06, 
           'min_child_samples': 88, 
           'max_bin': 380, 
           'od_type': 'IncToDec'
           }


In [21]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['INVC_CONT']):
  folds.append((train_idx,valid_idx))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=10.



train 데이터를 K-Fold 하여 각 Fold의 학습 값을 가지고 test 예측값을 구한 후
평균을 구하였습니다.

In [22]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(train, train['INVC_CONT']):
  folds.append((train_idx,valid_idx))

random.seed(42)
cat_models={}

cat_features =range(X_test.shape[1])

for fold in range(10):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = train.drop(['INVC_CONT'],axis=1).iloc[train_idx]
  X_valid = train.drop(['INVC_CONT'],axis=1).iloc[valid_idx]
  y_train = train['INVC_CONT'][train_idx].values
  y_valid = train['INVC_CONT'][valid_idx].values

  cat = CatBoostRegressor(**cat_param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_models[fold] = cat
  print(f'================================================================================\n\n')

====================================1============================================


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=10.



0:	learn: 5.6240726	test: 5.6244095	test1: 6.7835557	best: 6.7835557 (0)	total: 670ms	remaining: 42m 20s
100:	learn: 5.4960942	test: 5.5301616	test1: 6.6675145	best: 6.6675145 (100)	total: 25.7s	remaining: 15m 38s
200:	learn: 5.4405946	test: 5.4798163	test1: 6.6059558	best: 6.6059558 (198)	total: 40.6s	remaining: 12m 5s
300:	learn: 5.4140537	test: 5.4605846	test1: 6.5849293	best: 6.5849293 (300)	total: 53.7s	remaining: 10m 22s
400:	learn: 5.3916725	test: 5.4422328	test1: 6.5653535	best: 6.5653535 (400)	total: 1m 6s	remaining: 9m 20s
500:	learn: 5.3395210	test: 5.3956074	test1: 6.5185137	best: 6.5185137 (500)	total: 1m 22s	remaining: 9m 5s
600:	learn: 5.2901206	test: 5.3652920	test1: 6.4707527	best: 6.4707527 (600)	total: 1m 51s	remaining: 9m 50s
700:	learn: 5.0282278	test: 5.2082914	test1: 6.2761017	best: 6.2761017 (700)	total: 2m 19s	remaining: 10m 16s
800:	learn: 4.7363885	test: 5.0484783	test1: 6.1402382	best: 6.1401487 (798)	total: 2m 52s	remaining: 10m 43s
900:	learn: 4.5672517	te

In [23]:
submission.loc[:,'INVC_CONT']=0
for fold in range(10):
  submission.loc[:,'INVC_CONT'] += cat_models[fold].predict(test)/10

train을 K-fold한 값의 평균을 구하다 보니 예측값의 극단값이 작아질 수 밖에 없습니다.

따라서 INVC_CONT가 30 이상인 값에 적당한 값을 곱하여서 조정하였습니다.

In [24]:
submission.loc[submission.INVC_CONT>30,'INVC_CONT']=submission.loc[submission.INVC_CONT>30,'INVC_CONT']*4.8
submission.to_csv('/data/submission/마지막 확인용.csv',index = False)